In [10]:
import pickle
from operator import itemgetter
import pandas as pd
import plotly.graph_objects as go

from hydra.plotly_charts import PlotlyPriceChart, supersample_data
from hydra.utils import split_dupes


SYMBOLS = ["circle", "square", "diamond-tall", "diamond-wide"]
COLORS = ["red", "blue", "lime", "goldenrod"]
HEATMAP_COLORS = 'afmhot_r'
filename = "../2020-05-01 1d count=365 overlap=18h detrend=True.enviro"


file = None
with open(filename, "rb") as handle:
    file = pickle.load(handle)
interesting, charts, aggregate, inputs = itemgetter('data', 'charts', 'aggregate', 'inputs')(file)
timecap = pd.to_timedelta(inputs['timecap'])
overlap = pd.to_timedelta(inputs['overlap'])
traceCount = min(int(1.0/((timecap-overlap)/timecap)), len(charts))
kept_frequencies = aggregate['values']
# print(kept_frequencies)
complexity = aggregate['normalized']
 

pair = inputs['pair']
startDate = charts[0]['startDate']
endDate = charts[-1]['endDate']
# startDate = "2017-05-15 00:00"
# endDate = "2020-05-16 00:00"
print(startDate, endDate)
chart_data = {i:{
    "trendline":[],
    "ifft_price":[],
    "extrapolated_up": [],
    "extrapolated_down": [],
    "extrapolated_color": COLORS[i],
    "startDate":[],
    "endDate": []
} for i in range(traceCount)}

for index, chart in enumerate(charts):
    idx = index % traceCount
    chart_data[idx]['trendline'].append(chart['trendline'])
    chart_data[idx]['ifft_price'].append(chart['approximated_price'])
    chart_data[idx]['extrapolated_up'].append(chart['extrapolated_up'])
    chart_data[idx]['extrapolated_down'].append(chart['extrapolated_down'])
    chart_data[idx]['startDate'].append(chart['startDate'])
    chart_data[idx]['endDate'].append(chart['endDate'])
    

# print(trendlines)
priceChart = PlotlyPriceChart(pair, startDate, endDate)

def add_trace(data, chartParams={}, interval=1, skip_supersample=False):
    if skip_supersample:
        priceChart.add_trace( 
            go.Scatter(x=data.index, y=data.values, **chartParams), 
        )
        return

    supersampled = supersample_data(data, interval)
    priceChart.add_trace( 
        go.Scatter(x=supersampled[1440].index, y=supersampled[1440].values, **chartParams), 
        data=supersampled, 
    )


for index in range(traceCount):
    if index in chart_data:
        data = pd.concat(chart_data[index]['ifft_price'])
        add_trace(data, chartParams={"mode":"lines", "name":f"ifft-{index}"})
for index in range(traceCount):
    if index in chart_data:
        data = pd.concat(chart_data[index]['trendline'])
        add_trace(data, chartParams={"mode":"markers", "name":f"trend-{index}", "marker_symbol":SYMBOLS[index]})
for index in range(traceCount):
    if index in chart_data:
        data = pd.concat(chart_data[index]['extrapolated_up'])
        add_trace(data, chartParams={
            "mode":"markers", 
            "name":f"extra-up-{index}", 
            "marker_symbol": "triangle-up", 
            "marker_size":10, 
            "line_color":chart_data[index]["extrapolated_color"],
            # "text": chart_data[index]['endDate']
        }, skip_supersample=True)
        data = pd.concat(chart_data[index]['extrapolated_down'])
        add_trace(data, chartParams={
            "mode":"markers", 
            "name":f"extra-down-{index}", 
            "marker_symbol": "triangle-down", 
            "marker_size":10, 
            "line_color":chart_data[index]["extrapolated_color"],
            # "text": chart_data[index]['endDate']
        }, skip_supersample=True)

priceChart.add_trace(
    go.Heatmap(z=complexity.values.tolist(), x=complexity.columns.tolist(), y=complexity.index.tolist(), showscale=False), 
    loc=(1,1),
    traceArgs={}
)
    # priceChart.figure.add_vline(x=startDate, line_width=3)

priceChart.render()


2020-05-01 00:00:00 2020-07-31 23:59:00


FigureWidget({
    'data': [{'close': array([ 9004.7    ,  8824.6    ,  8854.9    ,  8983.1    ,  9052.6    ,
…